In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots

from lib import BlackFridayDataset

pd.options.plotting.backend = 'plotly'
pd.options.display.max_columns = None

In [3]:
dataset = BlackFridayDataset('data/train.csv', test_path='data/test.csv')
df, df_test = dataset.get_dfs()
# df_encoded, _ = dataset.preprocess_dfs(return_res=True)

In [ ]:
cols_to_drop = ['User_ID', 'Product_ID']
target_col = 'Purchase'
product_category_cols = ['Product_Category_1', 'Product_Category_2', 'Product_Category_3']
feature_cols = ['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status']

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df.isna().sum()

## Check that unique values of all features do not differ between train and test
This is necessary for data encoding. If the test dataset included values not seen in the train dataset, encoding would fail.

In [ ]:
cols_to_compare = df_test.drop(columns=cols_to_drop + product_category_cols).columns

In [ ]:
for col in cols_to_compare:
    assert set(df[col].unique()) == set(df_test[col].unique())

In [ ]:
df_categories = np.unique(df[product_category_cols].values.flatten())
df_categories = set(df_categories[~np.isnan(df_categories)])

df_test_categories = np.unique(df_test[product_category_cols].values.flatten())
df_test_categories = set(df_test_categories[~np.isnan(df_test_categories)])

assert len(df_test_categories - df_categories) == 0

In [ ]:
print(df_categories - df_test_categories)
print(df_test_categories - df_categories)

There are two missing product categories in the test dataset, but all of the product categories in the test dataset are present in the train dataset, hence this will not cause any problems with data encoding.

## Correlation between the feature variables and the target variable

Since our feature variables are discrete and unordered (except for possibly `Age` and `Stay_In_Current_City_Years`), traditional correlation methods are not suitable. For example, Pearson's correlation assumes that the data are continuous, while Spearman's correlation assumes order in the categorical data.

Instead, we measure the correlations using the $\eta^2$ method based on the ANOVA test, outputting a coefficient in the range of 0 and 1 with higher value indicating stronger association. 

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

cols = feature_cols + product_category_cols

df_correlations = pd.DataFrame(columns=['Feature', 'Correlation'], index=range(len(cols)))

for i, feature in enumerate(feature_cols + product_category_cols):
    df_rel = df[[feature, 'Purchase']]
    model = ols(f'Purchase ~ C({feature})', data=df_rel).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    eta_squared_category1 = anova_table['sum_sq'][f'C({feature})'] / anova_table['sum_sq'].sum()
    df_correlations.iloc[i] = [feature, eta_squared_category1]

df_correlations = df_correlations.sort_values(by='Correlation', ascending=False)

In [ ]:
df_correlations

There don't seem to be significant correlations between the demographic features and the purchase amount, while there is some level of association between the product categories and the target variable. This result is to be investigated further as we don't have much insight into the data, and knowing whether there is an ordinal relationship among the categories will help us choose the correct way to encode it, hence improving the model's performance.

Nevertheless, all of the features will be studied more thoroughly to further understand the data to help confirm or reject the correlation analysis.

## User ID

The `User_ID` column serves as a unique identifier of a user. It will not be used as an input for the model, however it can still offer interesting insights into the data.

TBD

In [ ]:
users_purchase_count = df.groupby(by='User_ID')['Purchase']\
    .agg(['mean', 'std', 'count'])\
    .sort_values(by='count', ascending=False)

In [ ]:
users_purchase_count

## Product ID

The `Product_ID` serves as a unique identifier for each product. It will not be used as an input feature. However, we can use it to better understand the dataset.

The questions we want to answer are:
- Are product categories constant among all purchases of a single product?
- How much does the purchase amount vary for a single product among purchases?

### Product categories

In [ ]:
df[['Product_ID'] + product_category_cols]\
    .fillna(0)\
    .groupby(by='Product_ID')[product_category_cols]\
    .nunique()\
    .eq(1)\
    .all()\
    .all()

Product categories are constant in all of the purchase instances.

## Purchase

In [ ]:
df.groupby(by='Product_ID')['Purchase']\
    .nunique()\
    .eq(1)\
    .all()\
    .all()

Purchase values are not constant among purchase instances. 

Let's investigate the differences in the most purchased products (for the sake of readability of the plot, we limit the amount of products to 9).

In [ ]:
n_products = 9

products_purchase_count = df.groupby(by='Product_ID')['Purchase']\
    .agg(['mean', 'std', 'count'])\
    .sort_values(by='count', ascending=False)

In [ ]:
fig = px.box(
    df[df['Product_ID'].isin(products_purchase_count.head(n_products).index)],
    x='Product_ID',
    y='Purchase',
)

fig.update_layout(
    xaxis_title='Product ID of the most purchased products',
    yaxis_title='Distribution of purchase amounts',
)

fig.show()

In [ ]:
fig_n_cols = 3
fig_n_rows = n_products // fig_n_cols + int(n_products % fig_n_cols != 0)

fig = make_subplots(
    rows=fig_n_rows,
    cols=fig_n_cols,
    x_title='Purchase amount',
    y_title='Number of occurrences',
    subplot_titles=products_purchase_count.head(n_products).index.tolist(),
    vertical_spacing=0.1,
)

for row_id in range(n_products):
    product_id = products_purchase_count.index[row_id]

    row = row_id // fig_n_cols
    col = row_id % fig_n_cols

    subplot = px.histogram(
        df[df['Product_ID'] == product_id]['Purchase'],
        nbins=100,
    )

    fig.add_trace(subplot.data[0], row=row + 1, col=col + 1)

fig.update_layout(showlegend=False, height=fig_n_rows * 300)

fig.show()

The data show that there are heavy differences in the purchase amount for all of the products, and the purchase amounts of a product seem to follow somewhat normal distributions around 5 mean values. This could be explained by two reasons:
- The different groups of purchase amounts exist because of discounts. Unfortunately, from the provided data, we have no way of checking whether this is true.
- The purchase amounts differ based on the users' demographics. We investigate this behavior by the following.
    - We compare users of the same demographics to the purchase value on the most purchased products.
    - For users that bought several products, we check whether they always belong to the same price group.

### Users of the same demographics

In [ ]:
def get_demographics_per_product(product_id: str) -> pd.DataFrame:
    df_rel = df[df['Product_ID'] == product_id]

    df_product_demographics = df_rel.copy()
    df_product_demographics = df_product_demographics[feature_cols + ['Purchase']]

    most_occuring_demographics = df_product_demographics.groupby(by=feature_cols)['Purchase'].count().sort_values(ascending=False).head(5).index

    df_product_demographics = df_product_demographics.set_index(feature_cols)
    df_product_demographics = df_product_demographics.loc[most_occuring_demographics, :]
    df_product_demographics = df_product_demographics.sort_index()

    for group, demographics in enumerate(most_occuring_demographics):
        df_product_demographics.loc[demographics, 'Demographics group'] = group

    df_product_demographics['Demographics group'] = df_product_demographics['Demographics group'].astype(int)

    return df_product_demographics

In [ ]:
df_product_demographics = get_demographics_per_product(products_purchase_count.index[1])

df_product_demographics.groupby(by='Demographics group')['Purchase'].agg(['mean', 'std', 'count'])

We substitute 5 of the most common demographic vectors with an integer representing the different groups.

There are only a couple of users sharing the same demographics and their purchase amounts don't seem to differ heavily at the first glance. 

In [ ]:
fig = px.histogram(df_product_demographics, x='Purchase', color='Demographics group')
fig.update_layout(xaxis_title='Purchase amount', yaxis_title='Number of occurrences')

fig.show()

The representation of different demographic groups is somewhat uniform among the purchase amounts, however since only a very small number of users share the same demographics, the result isn't of large significance. Hence, we instead compare the demographic features to the purchase amounts separately for a single product.

In [ ]:
fig_n_cols = 2
n_features = len(feature_cols)

fig = make_subplots(
    rows=n_features // fig_n_cols + int(n_features % fig_n_cols != 0),
    cols=fig_n_cols,
    subplot_titles=feature_cols,
    vertical_spacing=0.1,
    horizontal_spacing=0.05,
    x_title='Purchase amount',
    y_title='Number of occurrences',
)

product_id = products_purchase_count.index[0]
df_rel = df[df['Product_ID'] == product_id]

for ith_feature in range(n_features):
    feature = feature_cols[ith_feature]

    row = ith_feature // fig_n_cols
    col = ith_feature % fig_n_cols

    subplot = px.histogram(
        df_rel,
        x='Purchase',
        nbins=100,
        category_orders={feature: sorted(df_rel[feature].unique())},
        color=feature,
    )

    for trace_index, trace in enumerate(subplot.data):
        trace.name = f"{feature}: {trace.name}"
        fig.add_trace(trace, row=row+1, col=col+1)

fig.update_layout(height=fig_n_rows * 300)

fig.show()

There does not seem to be a significant relationship between any of the features and the purchase amounts, which confirms the results of our correlation analysis.

### Users' purchase amount group

Since the analysis of demographic features did not explain the purchase amounts groups, we further investigate by analysing whether the grouping of the users is consistent, i.e. whether one user always buys products for small prices while another buys the same products for higher prices. 

To do this, we assume that there are always 5 price groups, which is an observation that seems true at least for products with many purchases. We then cluster purchases of each product and compare users' distributions of the clusters.

We take into account only users with at least 100 purchases, and we eliminate products with less than 5 purchases to make the clustering possible.

In [ ]:
from sklearn.cluster import KMeans

df_clustered = df.copy()

product_counts = df_clustered.value_counts('Product_ID')
eligible_products = product_counts.index[product_counts.gt(5)]

df_clustered = df_clustered.loc[df_clustered['Product_ID'].isin(eligible_products)]
product_ids = df_clustered['Product_ID'].unique()

for product_id in product_ids:
    idx = df_clustered['Product_ID'] == product_id

    kmeans = KMeans(n_clusters=5, random_state=0)
    clusters = kmeans.fit_predict(df_clustered.loc[idx, ['Purchase']])

    centroids = kmeans.cluster_centers_.flatten()
    labels_ordered = np.argsort(centroids)
    centroids_ordered = centroids[labels_ordered]

    df_clustered.loc[idx, 'Cluster'] = clusters
    df_clustered.loc[idx, 'Cluster'] = df_clustered.loc[idx, 'Cluster'].map({k: v for k, v in enumerate(labels_ordered)})
    df_clustered.loc[idx, 'Cluster Centroid'] = df_clustered.loc[idx, 'Cluster'].map({k: v for k, v in enumerate(centroids_ordered)})

df_clustered['Cluster'] = df_clustered['Cluster'].astype(int)

In [ ]:
eligible_users = users_purchase_count[users_purchase_count['count'].ge(100)].index

In [409]:
df_clustered_eligible = df_clustered[df_clustered['User_ID'].isin(eligible_users)]
df_clustered_eligible.groupby('User_ID')['Cluster'].agg(['mean', 'std', 'count']).sort_values(by='mean', ascending=False)

,mean,std,count
User_ID,,,
1000307,2.771930,1.317245,114
1003101,2.761905,1.325892,147
1003864,2.753968,1.318705,126
1002185,2.742690,1.351858,171
1004801,2.732759,1.505686,116
...,...,...,...
1003610,1.796392,1.400364,388
1005558,1.788321,1.384954,137
1001495,1.778846,1.357873,104


In [ ]:
df_clustered_eligible['Cluster'].mean()

In [410]:
df_clustered_eligible.groupby('User_ID')['Cluster'].agg(['mean', 'std', 'count'])['mean'].hist()

## Investigate product categories

A product can fall into a minimum of one and a maximum of three categories, represented by columns `Product_Category_1`, `Product_Category_2` and `Product_Category_3`, where the second and third column have values missing. There are 20 different product categories, represented by integers from 1 to 20.

The missing values need to be handled before entering a model, and we need to investigate the features to be able to handle them correctly, without corrupting the data in any way.

We start by plotting the distribution of the number of categories a purchased product falls into. If most purchased products would fall into a maximum of, for example, 2 categories, we could simply decide to drop the mostly empty third column.

Note that each purchase of a product is considered in this statistic, hence a single product can contribute several times to the total. This makes sense because of how will the data be handled within the model.

The distribution of number of categories in purchased products looks as follows.

In [ ]:
fig = px.bar(
    df[product_category_cols].count(axis=1).value_counts(),
    title='Distribution of number of categories in purchased products',
)

fig.update_layout(
    xaxis_title='Number of categories',
    yaxis_title='Number of purchases',
    showlegend=False,
    xaxis_tickvals=[1, 2, 3]
)

fig.show()

The distribution is somewhat uniform, with 2 categories per purchased product being slightly dominant. This means that by dropping a column, we could lose a significant amount of information.

Another option is to fill the missing values with, for example, the most common value of that column. However, we don't know what do the values represent and if there's any relationship between them, hence filling the missing data with no further investigation could lead to biasing the dataset.

In [ ]:
df.groupby(by=product_category_cols)['Purchase'].agg('mean')

## Investigate users' purchasing behavior

## Investigate product-purchase relations

In [ ]:
df[product_category_cols].corrwith(df['Purchase'], method='spearman')

In [ ]:
df.groupby(by=product_category_cols[0])['Purchase'].agg(['mean', 'std', 'count'])